In [25]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import time
import cv2 as cv
import matplotlib.pyplot as plt
import os

Using matplotlib backend: QtAgg


## Cargamos el clasificador pre-entrenado

In [26]:
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier(r'C:\Users\Aulamultimedia\haarcascade_frontalface_alt.xml')
eye_cascade = cv.CascadeClassifier(r'C:\Users\Aulamultimedia\haarcascade_eye.xml')

## Hacemos el seguimiento de frames desde la webcam de la pc

In [55]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )
cap = cv.VideoCapture(0)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv.VideoWriter('pestaneo.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))


primera_lectura = True


while(1):

    ret ,frame = cap.read()


    if ret:
        key = False
        

        if key == True:
            break
        else:
            originalImage = frame
            # Imagen a escala de grises
            grayImage = cv.cvtColor(originalImage, cv.COLOR_BGR2GRAY)
            grayImage = cv.equalizeHist(grayImage)

            # Llamamos al clasificador de Haar (AdaBoost)

            faces = face_cascade.detectMultiScale(grayImage, 1.1, 1)

            # Recorro las caras encontradas
            #------------------------------
            for (x,y,w,h) in faces:
                cv.putText(originalImage, 'Presione \"q\" para salir',(80,450), cv.FONT_HERSHEY_PLAIN, 1.5,(0,255,255),2)
                # Le dibujamos un rectángulo rosa
                cv.rectangle(originalImage,(x,y),(x+w,y+h),(255,0,255),2)
                # Definimos las ROIs en la imagen gris y color
                roi_gray = grayImage[y:y+h, x:x+w] 
                roi_color = originalImage[y:y+h, x:x+w] 
                # Para cada rostro hallado le buscamos los ojos
                eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 8)
                # En los ojos hallados les dibujamos rectángulos azules
                for (ex,ey,ew,eh) in eyes:
                    cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)
                # Detecto si el programa está viendo los ojos
                if(len(eyes)>=2):
                    if (primera_lectura):
                        cv.putText(originalImage, 'Ojos detectados, \"s\" para empezar',(20,80), cv.FONT_HERSHEY_PLAIN, 2,(255,255,0),2)
                    else: 
                        cv.putText(originalImage, 'Ojos abiertos',(80,80), cv.FONT_HERSHEY_PLAIN, 2,(0,255,0),2)
                else:
                    if (primera_lectura):
                        cv.putText(originalImage, 'No se detectan los dos ojos',(10,80), cv.FONT_HERSHEY_PLAIN, 2,(255,255,0),2)
                    else: 
                        cv.putText(originalImage, 'pestaneo detectado',(80,80), cv.FONT_HERSHEY_PLAIN, 3,(0,255,0),2)
                        #cv.waitKey(3000)
                        #primera_lectura=True
            cv.imshow('Deteccion de pestaneos!',originalImage)
            w = cv.waitKey(1)
            if(w==ord('q')):
                break
            elif(w==ord('s') and primera_lectura):
                #This will start the detection
                primera_lectura = False
                
            out.write(frame)

    else:
        break


cap.release()
out.release()
cv.destroyAllWindows()